# Cloud Examples

The following code explores the result of over-the-counter models on the cloud. Results from the local semantic segmentation are comparable to these models and give more flexibility when cropping. These models where not used in the final implementation because they just provide bounding voices.

In [ ]:
import os
from PIL import Image, ImageDraw

image_name = 'example_1.jpg'

# Add known path to the image.
image_path = f"images/original/{image_name}"
# Clean name of image without the extension.
clean_image_name = image_name.split('.')[0]

with open(image_path, 'rb') as image_file:
    image_bytes = image_file.read()

# Normalize AWS Rekognition response
def normalize_aws(aws_objects):
    normalized_aws = []
    for obj in aws_objects:
        for instance in obj.get('Instances', []):
            normalized_obj = {
                'name': obj['Name'],
                'bounding_poly': {
                    'normalized_vertices': [
                        {'x': instance['BoundingBox']['Left'], 'y': instance['BoundingBox']['Top']},
                        {'x': instance['BoundingBox']['Left'] + instance['BoundingBox']['Width'], 'y': instance['BoundingBox']['Top']},
                        {'x': instance['BoundingBox']['Left'] + instance['BoundingBox']['Width'], 'y': instance['BoundingBox']['Top'] + instance['BoundingBox']['Height']},
                        {'x': instance['BoundingBox']['Left'], 'y': instance['BoundingBox']['Top'] + instance['BoundingBox']['Height']}
                    ]
                }
            }
            normalized_aws.append(normalized_obj)
    return normalized_aws

# Normalize Google Vision response
def normalize_google(google_objects):
    normalized_google = []
    for obj in google_objects:
        vertices = obj.bounding_poly.normalized_vertices
        normalized_obj = {
            'name': obj.name,
            'bounding_poly': {
                'normalized_vertices': [
                    {'x': vertices[0].x, 'y': vertices[0].y},
                    {'x': vertices[1].x, 'y': vertices[1].y},
                    {'x': vertices[2].x, 'y': vertices[2].y},
                    {'x': vertices[3].x, 'y': vertices[3].y}
                ]
            }
        }
        normalized_google.append(normalized_obj)
    return normalized_google

# Draw bounding boxes with the normalized objects
def draw_bounding_boxes(image_path, objects, title):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Add title to the image
    title_text = title
    title_text_size = draw.textbbox((10, 10), title_text)
    title_position = (10, 10)
    draw.rectangle([title_position, (title_text_size[2] + 10, title_text_size[3] + 10)], fill='red')
    draw.text(title_position, title_text, fill='white')

    for obj in objects:
        # Add object bounding boxes to the image
        box = [(vertex['x'] * image.width, vertex['y'] * image.height) for vertex in obj['bounding_poly']['normalized_vertices']]
        draw.line(box + [box[0]], width=3, fill='red')
        text = obj['name']
        text_bbox = draw.textbbox((box[0][0], box[0][1]), text)
        text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
        text_background = [(box[0][0], box[0][1] - text_height), (box[0][0] + text_width, box[0][1])]
        draw.rectangle(text_background, fill='red')
        draw.text((box[0][0], box[0][1] - text_height), text, fill='white')

    save_path = f'images/{title.lower()}/{os.path.basename(image_path)}'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    image.save(save_path)
    # Open the file you just saved.
    os.system(f'open {save_path}')

## Rekognition AWS

In [ ]:
import boto3

# Initialize the boto3 client
rekognition_client = boto3.client('rekognition')

def detect_objects():
    response = rekognition_client.detect_labels(
        Image={'Bytes': image_bytes},
        MaxLabels=40,
        MinConfidence=60
    )

    return response

aws_objects = detect_objects()['Labels']
# Make the output uniform to avoid having two drawing functions.
normalized_aws_objects = normalize_aws(aws_objects)

draw_bounding_boxes(image_path, normalized_aws_objects, 'AWS')

## Cloud Vision Google

In [ ]:
from google.cloud import vision

# Initialize the Vision client
client = vision.ImageAnnotatorClient()

def detect_objects():    
    image = vision.Image(content=image_bytes)
    response = client.object_localization(image=image)
    objects = response.localized_object_annotations

    return objects

google_objects = detect_objects()
# Make the output uniform to avoid having two drawing functions.
normalized_google_objects = normalize_google(google_objects)

draw_bounding_boxes(image_path, normalized_google_objects, 'Google')